## This notebook is to load the annotated dataset and work with it.

Input: a EHR note (MIMIC)

Instruction: Please identify 5~10 word tokens from the EHR note. Those 5~10 word tokens should be most important for a patient to understand their clinical conditions, procedures, and treatment plans.

Output: keywords [use either human annotated, or just use MIMIC outputs
[overlapping keywords between notes and discharge instructions- advantage: llama and GPT4]

## ======================================= Process annotation notes

## ========================================== Process with Rankings
### creating 5 fold CV datasets
- I will split 104 datasets into 5 folds (21,21,21,21,20)
- save the dataset

In [20]:
import pandas as pd
import random
random.seed(52)

pages = ['victoria', 'jiaping', 'jinying-3A', 'jinying-2B-part1', 'jinying_2B_part2','jinying_3C']

datas = []
for page in pages :
    # df = pd.read_excel("../data/raw/all_20160531.xlsx", page)
    df = pd.read_excel("../data/processed/all_20160531_changed.xlsx", page)
    datas.append(df)

In [21]:
def preprocess_data(data, topn, ids = []) :
    data = data.dropna(subset=['ranking']).copy()
    # data = data.drop(columns = ['packet'])

    data['fileid'] = data['fileid'].str.lower()
    data['fileid'] = data['fileid'].str.strip()
    data['fileid'] = data.fileid.str.replace('heart failure', 'heart_failure')
    data['fileid'] = data.fileid.str.replace('liver failure', 'liver_failure')
    data['fileid'] = data['fileid'].str.replace(" ", "")
    data['fileid'] = data['fileid'].apply(lambda x : x + '.txt' if not x.endswith('.txt') else x)

    # process phrases
    data['phrase'] = data['phrase'].str.lower()
    data['phrase'] = data['phrase'].str.strip()

    # process colors 
    data['color'] = data['color'].str.lower()
    data['color'] = data['color'].replace({"yellow" : "y", "green" : "g"})

    data = data.dropna()

    # symptoms
    symptoms = data[data.color == 'y'].sort_values(by=['fileid', 'ranking']).groupby(['fileid','color']).head(topn)
    symptoms = symptoms.drop_duplicates(subset=['fileid', 'phrase'])

    # tests
    labtests = data[data.color == 'g'].sort_values(by=['fileid', 'ranking'])
    labtests = labtests[labtests.ranking < (topn + 1)].copy()

    result = pd.concat([symptoms, labtests], ignore_index=True)
    result = result.sort_values(by=['fileid','ranking'])

    # grouped = data.sort_values(by=['fileid','ranking'])
    # result = grouped.groupby(['fileid']).head(topn)

    result = result.drop_duplicates(subset=['fileid', 'ranking'])
    result = result[~result.fileid.isin(ids)].reset_index(drop=True)
    return result.reset_index(drop=True), result['fileid'].unique().tolist()

In [22]:
# don't erase this!
example_text = "cancer.report24.txt"
example_texts = ["cancer.report24.txt", "diabetes.report310966.txt"]

In [23]:
fileids = pd.concat(datas).fileid.unique().tolist()

In [24]:
def split_cv(ids : list, nfold : int) :
    random.shuffle(ids)
    N = len(ids)

    counts = N // nfold
    leftover = N % nfold

    indices = [counts for idx in range(1,nfold+1)]
    indices = [idx + 1 if i < leftover else idx for i, idx in enumerate(indices)]

    folds = []
    past_idx = 0
    for i, idx in enumerate(indices) :
        idx += past_idx
        if i == len(indices) :
            folds.append(ids[past_idx:])
        else :
            folds.append(ids[past_idx:idx])
        past_idx = idx

    return folds

In [25]:
ids = []
processed_datasets_top10 = []
for data in datas :
    processed, data_ids = preprocess_data(data, 10, ids)
    processed_datasets_top10.append(processed)
    ids.extend(data_ids)

top10_dataset = pd.concat(processed_datasets_top10, ignore_index=True)
top10_dataset = top10_dataset[~(top10_dataset.fileid.isin(example_texts))].reset_index(drop=True)

In [36]:
top10_dataset.head()

,fileid,phrase,color,ranking
0,cancer.report28.txt,rheumatoid arthritis,y,1.0
1,cancer.report28.txt,rituximan,g,1.1
2,cancer.report28.txt,osteoporosis,y,2.0
3,cancer.report28.txt,denosumab,g,2.1
4,cancer.report28.txt,hypercalcemic,g,2.2


In [26]:
fileids = top10_dataset.fileid.unique().tolist()
cv5 = split_cv(fileids, 5)

In [38]:
"hello".replace("el", "")

'hlo'

Now merge with notes

In [27]:
notes = pd.read_pickle("../data/processed/notes.pkl")
notes.head()

,category,noteid,text
0,liver_failure,liver_failure.report37286.txt,This is a 50-year-old male with a history of d...
1,liver_failure,liver_failure.report37775.txt,Dr. name has discussed these results with you....
2,liver_failure,liver_failure.report38874.txt,"F/u on Osteoarthritis, chronic pain, HTN, Depr..."
3,liver_failure,liver_failure.report41972.txt,Very high a1c and glucose please follow up in ...
4,liver_failure,liver_failure.report51432.txt,name is a lovely just turned 65-year-old gentl...


In [29]:
import pickle

# the data are 104 dataset
files = top10_dataset['fileid'].unique()
# file_top5 = top3_dataset['fileid'].unique()

filtered_notes = notes[notes.noteid.isin(files)].reset_index(drop=True)

# ============== Description!! ============== #
# cv5 : 5 fold files
# top 10 dataset : ranking information of 104 datas
# filtered notes : the notes linked with top 10 dataset
processed_ranking_datasets = (cv5, top10_dataset, filtered_notes)

with open("../data/processed/cv_processed_ranking_datasets.pkl", 'wb') as f :
    pickle.dump(processed_ranking_datasets, f)

In [31]:
import json
from utils import format_prompt

def save_to_json(list_of_dicts, name) :
    with open(f"../data/processed/finetune/{name}.json", "w") as f :
        json.dump(list_of_dicts, f)


In [39]:
# * rule for naming
# * Name should be cv0_rank3, cv0_rank5, ...

for idx, cv in enumerate(cv5) : 
    for rank in [3,5,10] :
        output = format_prompt(cv, rank)
        name = f"cv{idx}_rank{rank}"
        save_to_json(output, name)

    # format_prompt(cv5[0], 3)

In [2]:
from datasets import Dataset
import pandas as pd
from utils import *

config = load_config()
PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/processed")
path = DATA_PATH.joinpath("cv_processed_ranking_datasets.pkl")
cv5, top10_dataset, filtered_notes = pd.read_pickle(path)

In [7]:
df = Dataset.from_pandas(filtered_notes)
df.filter(lambda x : x['noteid'] not in cv5[0])

Filter:   0%|          | 0/104 [00:00<?, ? examples/s]

Dataset({
    features: ['category', 'noteid', 'text'],
    num_rows: 83
})

In [8]:
top10_dataset

,fileid,phrase,color,ranking
0,cancer.report28.txt,rheumatoid arthritis,y,1.0
1,cancer.report28.txt,rituximan,g,1.1
2,cancer.report28.txt,osteoporosis,y,2.0
3,cancer.report28.txt,denosumab,g,2.1
4,cancer.report28.txt,hypercalcemic,g,2.2
...,...,...,...,...
1137,cancer.report3.txt,dystonic reaction,y,3.0
1138,cancer.report3.txt,reglan,g,3.1
1139,cancer.report3.txt,ivig,y,4.0
1140,cancer.report3.txt,rigors and difficulty breathing with ivig,g,4.1
